In [1]:
# 알고리즘 관련 라이브러리들
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


: 

: 

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np

In [5]:
# 가상환경 설정 ******
# myenv\Scripts\activate

# 데이터 로딩
raw_data = pd.read_excel('tomato.xlsx', skiprows=3, usecols="D:N", header=0, sheet_name=1)

In [6]:

# 엑셀 파일 전체 읽기
# raw_data = pd.read_excel('tomato.xlsx')

# 첫 번째 시트만 읽어오기 (인덱스 0)
# raw_data = pd.read_excel('tomato.xlsx', sheet_name=0)

# 3행을 건너뛰고 읽기 시작. 4번째 행을 컬럼명으로 사용.
# 열 이름으로 지정하여 가져올때 (열 이름이 'name', 'age', 'address'일 때) : usecols=['name', 'age', 'address']
# 열 인덱스로 지정 (0부터 시작) : usecols=[0, 2, 4]
# 열 범위로 지정 : usecols="D:N" (칼럼중 착과화방수, 수확화방수의 뜻이 모호해 데이터에 포함하지 않음)

raw_data = pd.read_excel('tomato.xlsx', skiprows=3, usecols="D:N", header=0, sheet_name=1)

In [7]:
#소수점 아래 원하는 자릿수만큼만 표시
pd.set_option('display.float_format', '{:.2f}'.format)

In [8]:
# 평균값이 들어가 있는 행을 삭제 : 평균의값의 의미가 모호함 , 마지막의 평균품종값도 제거하였음
raw_data.drop([3, 7, 11, 15, 16], axis=0, inplace=True)

In [9]:
# 처음부터 11행까지만 남기고 나머지는 삭제 : 데이터가 들어 있지 않은 행도 모두 표현되기 때문 
raw_data = raw_data.iloc[:12]
# raw_data의 값을 다시 재정렬
raw_data.reset_index(drop=True, inplace=True)

In [10]:
# 전체 행의 수를 출력해 제대로 정제 되었는지 확인 
print("전체 행의 수:", raw_data.shape[0])

전체 행의 수: 12


In [11]:
# 모든행을 출력하여 데이터가 df에 제대로 들어가 있는지 확인
# pd.set_option('display.max_rows', None)  # None으로 설정하면 모든 행을 출력
print(raw_data)

       ㎝     ㎜  ㎜.1   ㎝.1    ea  ㎝.2   ㎝.3   ㎝.4   ㎝.5  ea.1  ea.2
0  25.00 10.98 3.24 20.00 15.50 4.20 33.30 33.30 21.00 16.00 23.00
1  21.50 13.06 4.02 31.00    16 4.90 35.00 35.00 27.00 14.00 20.00
2  21.00  9.48 2.19 19.00 15.75 3.20 35.50 32.50 21.00 14.00 21.00
3  23.50 13.23 3.25 35.50    17 4.20 32.50 41.50 34.90 10.00 23.00
4  24.90 12.20 3.28 39.30 15.95 4.40 36.00 39.00 32.50 11.00 22.00
5  22.30 14.23 3.75 32.00    17 4.60 55.00 37.00 31.20 12.00 20.00
6  22.50 14.20 3.82 41.00 16.50 6.00 33.50 42.00 38.00 13.00 16.00
7  20.00 11.86 3.16 23.00    16 5.60 36.00 42.50 31.90 12.00 18.00
8  22.60 12.62 2.75 27.90 16.50 4.00 33.60 35.50 31.50 13.00 16.00
9  17.00 14.18 4.14 32.00 14.95 3.80 31.50 39.50 34.00 14.00 17.00
10 24.00 12.09 2.32 39.00 15.25 5.60 33.00 38.00 34.20 17.00 20.00
11 23.00 11.08 4.01 20.50 15.50 4.00 31.50 37.00 22.00 16.00 19.00


In [12]:
# cm와 mm 통일 : cm의 값이 많으므로 cm로 단위를 통일하였음
raw_data['㎜'] /= 10
raw_data['㎜.1'] /= 10

In [13]:
#칼럼들의 이름확인
print(raw_data.columns)

Index(['㎝', '㎜', '㎜.1', '㎝.1', 'ea', '㎝.2', '㎝.3', '㎝.4', '㎝.5', 'ea.1',
       'ea.2'],
      dtype='object')


In [14]:
# 칼럼의 이름을 알수 있게 변경 
# 초장 경경 화방경경 개화위치 개화수 화방줄기거리 화방간거리
# ㎝   ㎜   ㎜.1    ㎝.1     ea     ㎝.2       ㎝.3
# 엽장 엽폭 엽수    착과량      착과화방수 수확화방수
# ㎝.4 ㎝.5 ea.1    ea.2       ea.3      ea.4
new_column_names = {
    '㎝': 'cm_chojang_len', '㎜': 'cm_julgi_len', '㎜.1': 'cm_hwabang_len',
    '㎝.1': 'cm_gyahwa_loca','ea': 'cm_gyahwa_ea', '㎝.2': 'cm_hwabangJulgi_len', '㎝.3': 'cm_hwabanggan_len',
    '㎝.4': 'cm_yebjang_len','㎝.5': 'cm_yebjang_wid', 'ea.1': 'cm_yeb_ea',
    'ea.2': 'cm_gyasu_num',
    # 다른 이름들도 이런 식으로 변경
}
raw_data.rename(columns=new_column_names, inplace=True)

In [15]:
# 데이터의 상위 5개 행과 데이터의 정보를 출력
print(raw_data.head())

# 이후할일 : 이후에는 cm,mm 를 통일한다. 평균값을 어떻게 처리할지 생각해보자. 평균값은 정확하지 않은 관계로 일단 제외함.

   cm_chojang_len  cm_julgi_len  cm_hwabang_len  cm_gyahwa_loca cm_gyahwa_ea  \
0           25.00          1.10            0.32           20.00        15.50   
1           21.50          1.31            0.40           31.00           16   
2           21.00          0.95            0.22           19.00        15.75   
3           23.50          1.32            0.33           35.50           17   
4           24.90          1.22            0.33           39.30        15.95   

   cm_hwabangJulgi_len  cm_hwabanggan_len  cm_yebjang_len  cm_yebjang_wid  \
0                 4.20              33.30           33.30           21.00   
1                 4.90              35.00           35.00           27.00   
2                 3.20              35.50           32.50           21.00   
3                 4.20              32.50           41.50           34.90   
4                 4.40              36.00           39.00           32.50   

   cm_yeb_ea  cm_gyasu_num  
0      16.00         23.00 

In [16]:
print(raw_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cm_chojang_len       12 non-null     float64
 1   cm_julgi_len         12 non-null     float64
 2   cm_hwabang_len       12 non-null     float64
 3   cm_gyahwa_loca       12 non-null     float64
 4   cm_gyahwa_ea         12 non-null     object 
 5   cm_hwabangJulgi_len  12 non-null     float64
 6   cm_hwabanggan_len    12 non-null     float64
 7   cm_yebjang_len       12 non-null     float64
 8   cm_yebjang_wid       12 non-null     float64
 9   cm_yeb_ea            12 non-null     float64
 10  cm_gyasu_num         12 non-null     float64
dtypes: float64(10), object(1)
memory usage: 1.2+ KB
None


In [17]:
print(raw_data.columns)

Index(['cm_chojang_len', 'cm_julgi_len', 'cm_hwabang_len', 'cm_gyahwa_loca',
       'cm_gyahwa_ea', 'cm_hwabangJulgi_len', 'cm_hwabanggan_len',
       'cm_yebjang_len', 'cm_yebjang_wid', 'cm_yeb_ea', 'cm_gyasu_num'],
      dtype='object')


In [18]:
# x는 목표에 영향을 주는값들 , y는 우리가 목표로 하는값
# 그래서 토마토수확량이 목표라고 하면 토마토수확량의칼러명을 y로 지정하고
# 그외 토마토수확량에 영향을 주는 값들을 x로 지정해야된다.
# 그래서 아래코드에서 x에 y칼럼을 제외한 모든 값들이 들어가게 된것임. 결론족으로 두군데 다 목표칼럼이 들어가야됨 
# axis=1 은 컬럼 , axis=0은 행을 지정함 
X = raw_data.drop("cm_gyasu_num", axis=1)
y = raw_data["cm_gyasu_num"]

In [19]:
# 데이터를 학습용과 테스트용으로 분할. 테스트 데이터의 비율은 20%로 설정 *딥러닝관련 2,3 참고
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
import keras

ImportError: cannot import name 'deserialize_keras_object' from partially initialized module 'keras.saving.legacy.serialization' (most likely due to a circular import) (/Users/seungwoo/opt/anaconda3/envs/ngf/lib/python3.8/site-packages/keras/saving/legacy/serialization.py)

In [27]:
%pip install --upgrade keras tensorflow

  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/2e/f3/19da7511b45e80216cbbd9467137b2d28919c58ba1ccb971435cb631e470/keras-2.13.1-py3-none-any.whl.metadata
  Using cached keras-2.13.1-py3-none-any.whl.metadata (2.4 kB)
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/ee/28/f6658e1c660aa86b135c60e952007dc03b75ff40f0543ecd2dc5e16e763c/tensorflow-2.13.0-cp38-cp38-macosx_10_15_x86_64.whl.metadata
  Using cached tensorflow-2.13.0-cp38-cp38-macosx_10_15_x86_64.whl.metadata (3.2 kB)
  Obtaining dependency information for flatbuffers>=23.1.21 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Obtaining dependency information for tensorflow-estimator<2.14,>=2.13.0 

In [25]:
from tensorflow import keras

In [26]:
from tensorflow.python.keras import layers
from tensorflow.python.keras.models import Sequential

# 딥러닝 모델을 구성
model = tf.keras.Sequential([
    # 첫 번째 은닉층: 64개의 노드와 ReLU 활성화 함수를 사용
    tf.keras.layers.Dense(64, activation='relu', input_shape=[len(X_train.keys())]),
    # 두 번째 은닉층: 32개의 노드와 ReLU 활성화 함수를 사용
    tf.keras.layers.Dense(32, activation='relu'),
    # 출력층: 1개의 노드를 사용
    tf.keras.layers.Dense(1)
])

ImportError: cannot import name 'deserialize_keras_object' from partially initialized module 'keras.saving.legacy.serialization' (most likely due to a circular import) (/Users/seungwoo/opt/anaconda3/envs/ngf/lib/python3.8/site-packages/keras/saving/legacy/serialization.py)

In [19]:
# 모델을 컴파일. 손실 함수로는 MSE를 사용
model.compile(optimizer='adam', loss='mse')

In [20]:
#데이터 타입 확인 및 변환: X_train과 y_train이 float 타입인지 확인하고, 아니라면 float 타입으로 변환
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')

# 모델을 학습합니다. 학습 데이터 중 20%는 검증용으로 사용
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
1/1 [==============================] - 1s 651ms/step - loss: 3.9982 - val_loss: 19.1470
Epoch 2/50
1/1 [==============================] - 0s 25ms/step - loss: 12.3561 - val_loss: 15.7675
Epoch 3/50
1/1 [==============================] - 0s 26ms/step - loss: 4.1917 - val_loss: 23.5143
Epoch 4/50
1/1 [==============================] - 0s 24ms/step - loss: 5.3002 - val_loss: 29.1474
Epoch 5/50
1/1 [==============================] - 0s 24ms/step - loss: 8.0937 - val_loss: 25.4788
Epoch 6/50
1/1 [==============================] - 0s 24ms/step - loss: 6.1030 - val_loss: 19.3291
Epoch 7/50
1/1 [==============================] - 0s 23ms/step - loss: 3.4816 - val_loss: 16.0097
Epoch 8/50
1/1 [==============================] - 0s 25ms/step - loss: 3.4551 - val_loss: 15.6974
Epoch 9/50
1/1 [==============================] - 0s 25ms/step - loss: 5.0242 - val_loss: 15.8177
Epoch 10/50
1/1 [==============================] - 0s 23ms/step - loss: 5.4102 - val_loss: 15.6521
Epoch 11/50
1/1 [

In [21]:
# 모델을 테스트 데이터로 평가합
predictions = model.predict(X_test)

1/1 [==============================] - 0s 70ms/step


In [22]:
# RMSE 값을 계산
rmse = np.sqrt(mean_squared_error(y_test, predictions))

In [23]:
# R2 스코어를 계산
r2 = r2_score(y_test, predictions)

In [24]:
# 결과를 출력
print(f'RMSE: {rmse}')
print(f'R2 Score: {r2}')

RMSE: 2.0115852605293756
R2 Score: 0.3255874566034941
